### Imports


In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from sklearn.model_selection import LeaveOneGroupOut, KFold
from sklearn.preprocessing import LabelEncoder
from torchvision.models import mobilenet_v3_small
from tqdm import tqdm

from src.dataset_loaders import ISAdetectDataset
from src.transforms import GrayScaleImage

### Setup


In [2]:
# Specify the model
MODEL = mobilenet_v3_small
TARGET_FEATURE = "architecture"

# Model hyperparameters
BATCH_SIZE = 64
NUM_EPOCHS = 3
LEARNING_RATE = 1e-4

# Specify which groups to use as validation set. Set to None to validate all groups.
VALIDATION_GROUPS = None
# VALIDATION_GROUPS = ["mips", "mipsel"]

# Set to an integer to limit the dataset size. Set to None to disable limit.
MAX_FILES_PER_ISA = None

### Helper functions


In [3]:
def get_device():
    """
    Returns 'cuda' if CUDA is available, else 'mps' if Apple Silicon GPU is available,
    otherwise 'cpu'.
    """
    device = None
    if torch.cuda.is_available():
        device = torch.device("cuda")
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
    else:
        device = torch.device("cpu")

    print(f"Using device: {device}")
    return device

### Prepare


In [4]:
device = get_device()

dataset = ISAdetectDataset(
    dataset_path="../../dataset/ISAdetect/ISAdetect_full_dataset",
    transform=GrayScaleImage(224, 224),
    file_byte_read_limit=224 * 224,
    per_architecture_limit=MAX_FILES_PER_ISA,
)

groups = list(map(lambda x: x["architecture"], dataset.metadata))
target_feature = list(map(lambda x: x[TARGET_FEATURE], dataset.metadata))
print(f"groups: {set(groups)}")
print(f"features: {set(target_feature)}")

Using device: cuda
groups: {'ppc64', 'armhf', 'hppa', 'sh4', 's390x', 'm68k', 's390', 'alpha', 'i386', 'sparc', 'armel', 'x32', 'powerpc', 'amd64', 'ia64', 'mipsel', 'mips64el', 'powerpcspe', 'arm64', 'mips', 'ppc64el', 'riscv64', 'sparc64'}
features: {'ppc64', 'armhf', 'hppa', 'sh4', 's390x', 'm68k', 's390', 'alpha', 'i386', 'sparc', 'armel', 'x32', 'powerpc', 'amd64', 'ia64', 'mipsel', 'mips64el', 'powerpcspe', 'arm64', 'mips', 'ppc64el', 'riscv64', 'sparc64'}


### Train and evaluate


In [5]:
kfold = KFold(n_splits=10, shuffle=True)
label_encoder = LabelEncoder()

fold = 1
accuracies = {}
corrects = {}
totals = {}
losses_per_fold = []  # Will store losses for each fold
log_every_n_steps = 10
for train_idx, test_idx in kfold.split(X=range(len(dataset))):

    print(f"\n=== Fold {fold} ===")
    fold += 1

    all_train_labels = [dataset.metadata[i][TARGET_FEATURE] for i in train_idx]
    label_encoder.fit(all_train_labels)

    train_dataset = Subset(dataset, train_idx)
    test_dataset = Subset(dataset, test_idx)

    train_loader = DataLoader(
        train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8
    )
    test_loader = DataLoader(
        test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8
    )

    model = MODEL(num_classes=23, weights=None)
    model.features[0][0] = nn.Conv2d(
        1, 16, kernel_size=3, stride=2, padding=1, bias=False
    )

    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    current_fold_losses = []
    step_counter = 0
    # Train model
    for epoch in range(NUM_EPOCHS):
        model.train()
        print(f"Epoch {epoch+1}:")
        epoch_losses = []

        for images, labels in tqdm(train_loader):
            images = images.to(device)

            encoded_labels = torch.from_numpy(
                label_encoder.transform(labels[TARGET_FEATURE])
            ).to(device)

            optimizer.zero_grad()
            predictions = model(images)
            loss = criterion(predictions, encoded_labels)
            loss.backward()
            optimizer.step()

            epoch_losses.append(loss.item())
            if step_counter % log_every_n_steps == 0:
                current_fold_losses.append({"step": step_counter, "loss": loss.item()})
            step_counter += 1

        avg_epoch_loss = sum(epoch_losses) / len(epoch_losses)
        print(f"Epoch {epoch+1} Average Loss: {avg_epoch_loss:.4f}")
        print(f"Epoch {epoch+1} Final Loss: {epoch_losses[-1]:.4f}")

        # Evaluate model
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in test_loader:
                images = images.to(device)
                encoded_labels = torch.from_numpy(
                    label_encoder.transform(labels[TARGET_FEATURE])
                ).to(device)

                outputs = model(images)

                _, predicted = torch.max(outputs, 1)
                correct += (predicted == encoded_labels).sum().item()
                total += encoded_labels.size(0)

                for i in range(len(predicted)):
                    # add accuracy per architecture
                    arch = label_encoder.inverse_transform([encoded_labels[i].item()])[
                        0
                    ]
                    if predicted[i] == encoded_labels[i]:
                        corrects[arch] = corrects.get(arch, 0) + 1
                    totals[arch] = totals.get(arch, 0) + 1

        accuracy = correct / total
        print(f"Test Accuracy: {100*accuracy:.2f}%")

    accuracies[fold] = accuracy
    losses_per_fold.append({"fold": fold - 1, "losses": current_fold_losses})


=== Fold 1 ===
Epoch 1:


 35%|███▌      | 479/1356 [00:13<00:24, 36.40it/s]


KeyboardInterrupt: 

### Evaluate


In [ ]:
print("Test accuracies for each architecture:")
for arch, correct_count in corrects.items():
    print(f"{arch}: {100*correct_count/totals[arch]:.2f}%")


# Print overall performance across folds
mean_acc = np.mean(list(accuracies.values()))
std_acc = np.std(list(accuracies.values()))
print(f"\nAverage LOGO cross-validated test accuracy: {mean_acc:.4f} ± {std_acc:.4f}")

In [ ]:
for i, loss_data in enumerate(losses_per_fold):
    plt.plot(
        [x["step"] for x in loss_data["losses"]],
        [x["loss"] for x in loss_data["losses"]],
        label=f"Fold {loss_data['fold']} - {loss_data['group']}",
    )
    plt.show()